# Fake PTA tutorial

Fake PTA is a tool for basic Pulsar Timing Array (PTA) simulations designed to work with ENTERPRISE (https://github.com/nanograv/enterprise). It copies the main features of the Pulsar object in ENTERPRISE while allowing complete modification of its attributes, making it easier to play with pulsar properties. This tool allows to simulate pulsar post-fit timing residuals (timing model is not taken into account, for this, see libstempo at https://github.com/vallis/libstempo). In its current version, it simulates white noise (EFAC, EQUAD), red noise (RN), dispersion measure (DM) noise, scattering variation (Sv) chromatic noise and common red noises correlated between pulsars with various overlap reduction functions (Hellings-Down, monopole, dipole), as well as deterministic continuous gravitational wave (CGW) signals. Because the pulsar residuals are openly accessible, it is easy to inject any type of custom signal.

# Getting started

In [ ]:
from fakepta.fake_pta import Pulsar, copy_array, make_fake_array
from fakepta.correlated_noises import add_correlated_red_noise_gp